This function computes the **gravitational acceleration** experienced by each body
based on the position of the other. It uses Newton's inverse-square law:

$$
\vec{F} = -m1*m2\frac{\vec{r}}{|\vec{r}|^3}
$$

The output consists of two vectors: one for each mass.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def acceleration(r1, r2, m1, m2, G):
    """Compute gravitational accelerations on m1 and m2"""
    r = r2 - r1
    dist = np.linalg.norm(r)
    
    # Gravitational force vector (Newton's law)
    force = G * m1 * m2 * r / dist**3

    # Accelerations (a = F / m)
    acc1 = +force / m1
    acc2 = -force / m2

    return acc1, acc2

This function performs one step of the **Velocity-Verlet algorithm**, updating the
positions and velocities of both bodies. It is designed for systems where the
acceleration depends only on position (like gravity).

Use this function inside a loop to simulate the orbit over time.

In [ ]:
# HINT: Implement this in a loop
def velocity_verlet_step(r1, v1, r2, v2, dt):
    a1, a2 = acceleration(r1, r2)

    r1_new = r1 + v1 * dt + 0.5 * a1 * dt**2
    r2_new = r2 + v2 * dt + 0.5 * a2 * dt**2

    a1_new, a2_new = acceleration(r1_new, r2_new)

    v1_new = v1 + 0.5 * (a1 + a1_new) * dt
    v2_new = v2 + 0.5 * (a2 + a2_new) * dt

    return r1_new, v1_new, r2_new, v2_new

A standard 4th-order Runge–Kutta integration step for solving ODEs.
It requires defining your state vector `u` (e.g., positions and velocities concatenated)
and a derivative function `f(u)`.

Apply it inside a loop to advance your system in time.

In [ ]:
def rk4_step(f, u, dt):
    """Generic RK4 solver step"""
    k1 = f(u)
    k2 = f(u + 0.5 * dt * k1)
    k3 = f(u + 0.5 * dt * k2)
    k4 = f(u + dt * k3)
    return u + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

# HINT: define u = [r1, v1, r2, v2] flattened into 1D vector
# and f(u) to compute du/dt for all state variables

This snippet implements the **Forward Time Centered Space (FTCS)** method
for solving the 2D heat diffusion equation numerically.

The updated temperature field depends on the four neighbouring grid points
and the current temperature at each grid location.

In [ ]:
# Grid setup
Nx, Ny = 50, 50
T = np.zeros((Nx, Ny))
T[25, 25] = 100  # initial condition: hot spot in the middle

def ftcs_step(T, kappa, tau, h):
    T_new = T.copy()
    for j in range(1, Nx-1):
        for l in range(1, Ny-1):
            T_new[j,l] = T[j,l] + (kappa * tau / h**2) * (
                T[j+1,l] + T[j-1,l] + T[j,l+1] + T[j,l-1] - 4 * T[j,l]
            )
    return T_new

#### **Hints:**

- When doing von Neumann analysis, assume a Fourier-mode solution and substitute into your FTCS equation. Focus on the exponential growth factor and derive the condition that ensures it stays ≤ 1 in magnitude.
- Use `np.linalg.norm` to compute distances.
- For phase space, plot one component of position vs. one component of momentum or velocity.
- Don’t forget units: you may need to “un-normalise” when computing theoretical values for orbital periods.

## Plots

This snippet sets up two lists, `r1_traj` and `r2_traj`, to store the position vectors of the two bodies over time. These are used later to plot the orbits.

The function `total_energy(r1, v1, r2, v2)` calculates the total mechanical energy of the system at any given time step. It includes:

- **Kinetic energy** of both masses, using \(\frac{1}{2}mv^2\)
- **Potential energy** due to gravity, using \(-\frac{G m_1 m_2}{r}\)

This function is useful to monitor **energy conservation** in numerical simulations. Ideally, the total energy should remain nearly constant if the integration scheme is stable and accurate.

In [ ]:
r1_traj, r2_traj = [], []
energy = []

def total_energy(r1, v1, r2, v2):
    """Compute total energy of the system"""
    kinetic = 0.5 * m1 * np.linalg.norm(v1)**2 + 0.5 * m2 * np.linalg.norm(v2)**2
    potential = - (m1 * m2) / np.linalg.norm(r1 - r2)
    return kinetic + potential

This plotting code displays the temperature field `T` as a heatmap after evolving the system for a number of time steps using the FTCS method.

- `plt.imshow` visualises the 2D grid.
- The `'hot'` colormap is used to show temperature intensity.
- A colorbar is included to indicate the temperature scale.

This plot gives a clear visual indication of how the initial heat (e.g. from a central hot spot) diffuses across the grid.

In [ ]:
# --- Plotting ---
plt.figure(figsize=(6, 5))
plt.imshow(T, cmap='hot', origin='lower')
plt.colorbar(label='Temperature')
plt.title(f'2D Heat Diffusion after {n_steps} steps')
plt.xlabel('x')
plt.ylabel('y')
plt.show()